In [2]:
import pandas as pd
import numpy as np
import pickle
import time

structures = pd.read_csv('../Data/structures.csv')
train = pd.read_csv('../Data/test.csv')

In [30]:
symetry = '1JHN'
file_prefix = '../Data/Images/'+symetry+'/'
train = train[train['type'] == symetry]
import os
if not os.path.exists(file_prefix):
    os.makedirs(file_prefix)
print(f'selecting {train.shape[0]} bonds from {symetry}')

selecting 24195 bonds from 1JHN


In [4]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
d_bond = train_p_0 - train_p_1
nbonds = len(d_bond)

train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
center = (train_p_0 + train_p_1) / 2

we define a function that performs the integration along the circles:

In [5]:
def density_integration(df, nb_angles):
    density_integrale = []
    for icouple in range(df.shape[0]): 

        step_length = length[icouple]
        p = df[icouple, :, :].copy()   
        density = np.zeros(nb_angles)

        for iatom in range(len(atoms)):
            val = p - atoms[iatom]
            dist = np.linalg.norm(val, axis=1)
            density += 1 / dist**2

        integrale = density.sum() * step_length
        density_integrale.append(integrale)
    return density_integrale

We define images of size 35 x 35:

In [6]:
nb_r = 35
nb_R = 35
nb_angles = 45

values = [(r, R) for r in np.linspace(5, 0.01, nb_r) for R in np.linspace(-5, 5, nb_R)]

In [29]:
start_time = time.time()

image_file = 'images.pkl'
bond_file = 'bond.pkl'

#bond_list = []
#image_set = []

ids = train['id'].values
for ibond in range(nbonds): 
    #__________________
    d = d_bond[ibond]
    orig = center[ibond]

    molec = train.iloc[ibond]['molecule_name']
    #bond_list.append(molec+'_'+str(ids[ibond]))

    norm = np.linalg.norm(d)
    d = d / norm
    #______________________
    if d[2] != 0:
        u = np.ones(3)
        u[2] = -(d[0]*u[0] + d[1]*u[1]) / d[2]
        norm = np.linalg.norm(u)
        u = u / norm
    elif d[0] != 0:
        u = np.ones(3)
        u[0] = - d[1]*u[1] / d[0]
        norm = np.linalg.norm(u)
        u = u / norm
    else:
        u = np.ones(3)
        u[1] = 0
        norm = np.linalg.norm(u)
        u = u / norm
    #____________________________________
    v = np.cross(d, u)
    norm = np.linalg.norm(v)
    v = v / norm
    #_______________________________
    new_data = []
    length = []
    for r, R in values:
        center_circle = R * d  + orig 
        #______________________________________________________________
        val = []
        for theta in np.linspace(0, 2 * np.pi, nb_angles):
            xp = r * np.cos(theta)
            yp = r * np.sin(theta)
            val.append([xp, yp])
    #         print(theta)
        #________________________________
        new_pt = []
        for pt in val:
            new_pt.append(center_circle + (pt[0] * u + pt[1] * v ))
        new_data.append(np.array(new_pt))

        length.append(r * (2 * np.pi) / (nb_angles - 1))
    #__________________________________
    new_data = np.array(new_data)
    #__________________________________
    image = []
    for atom in ['C', 'N', 'O', 'H', 'F']:
        atoms = structures[(structures['molecule_name'] == molec) &
                           (structures['atom'] == atom)][['x', 'y', 'z']].values

        data = density_integration(new_data, nb_angles)
        image.append(np.array(data).reshape(nb_r, nb_R))

    #image_set.append(image)
    
    with open(file_prefix+molec+'_'+str(ids[ibond])+'.pkl', 'wb') as fp:
        pickle.dump(image, fp)

# with open(bond_file, 'wb') as fp:
#     pickle.dump(bond_list, fp)
        
time.time() - start_time

5.810262680053711